# Char level transformer for Machine translation
To do translation, we'll use sequence-to-sequence model: an encoder encode the input and decoder using both the encode and raw output to predict. 

This is similar to char-transformer for language modeling, but we will add an encoder, and in place of top-k output when we generate text we'll use top-1 beam search output.

<img src='images/transformer.png' width=400>
<a href="https://www.kaggle.com/hungnm/englishvietnamese-translation">English-Vietnamese dataset</a>

In [2]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

import logging
logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
)

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
class CharDataset(Dataset):
    '''
    Dataset is a iterable that returns input and target sentence. It adds <sos> at the begining, and <eos> at the end, 
    and filling in <pad> if sentence length is less than pre-defined value.
    '''
    def __init__(self, x, y, sequence_len):
        # data in the type of pairs of sentence

        data = ''.join(x+y)
        chars = ['<pad>'] +['<sos>'] + ['<eos>'] + sorted(list(set(data)))
        data_size, vocab_size = len(data), len(chars)

        print('data has %d characters, %d unique chars, %d sentences.' % (data_size, len(chars), len(x)))
        print('%d sentence length' % (sequence_len))
        
        self.x, self.y = x, y
        self.ch2i = {ch:i for i,ch in enumerate(chars)}
        self.i2ch = {i:ch for i,ch in enumerate(chars)}
        self.vocab_size = vocab_size
        self.sequence_len = sequence_len
    
    def __len__(self):
        return len(self.x) # len x = y
    
    def __getitem__(self, idx):
        indx = self.padding([self.ch2i[ch] for ch in self.x[idx]] + [self.ch2i['<eos>']])
        indy = [self.ch2i['<sos>']] + self.padding([self.ch2i[ch] for ch in self.y[idx]] + [self.ch2i['<eos>']])

        x = torch.tensor(indx, dtype=torch.long)
        y = torch.tensor(indy, dtype=torch.long)
                                                                                                                               
        return x,y
                                                                                                                               
    def padding(self, string):
        if len(string)<self.sequence_len:
            string =  string + [0]*(self.sequence_len - len(string))
        else:
            string = string[:self.sequence_len -1] + [self.ch2i['<eos>']]
                   
        return string
                                                                                                                               


In [4]:
# walk through data folder
import os
text = {}
for (dirpath, dirnames, filenames) in os.walk("data"):
    for name in filenames:
        path = os.path.join(dirpath, name)
        with open(path, encoding='utf-8') as f:
            text[name] = f.read().lower().split('\n')

x,y = text['vi_sents'], text['en_sents']

# print some samples
for i in range(-1,-3,-1):
    print(x[i],'|',y[i])

bạn đã ở đám cưới của tôi | you were at my wedding
tại sao bạn không quan tâm? | why aren't you interested?


In [5]:
nine_nine_percentile = int(np.percentile([len(sen) for sen in x],99))
dataset = CharDataset(x, y, sequence_len=nine_nine_percentile)

print('sample tensors ', next(iter(dataset)))
print("vocab: ", dataset.ch2i)

data has 17367685 characters, 151 unique chars, 254090 sentences.
81 sentence length
sample tensors  (tensor([ 51,  36,  41,   3,  49,  48,  36,   3,  39,  71,  41,  34,   3,  82,
        103,  47,   3,  41,  34,  86, 122,  36,   3,  44,  48,  65,  47,   3,
         45,  60,  30,   3,  47,  45,  42,  41,  34,   3,  47, 127,   3,  30,
         35, 118,  36,   2,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0]), tensor([ 1, 43, 39, 32, 28, 46, 32,  3, 43, 48, 47,  3, 47, 35, 32,  3, 31, 48,
        46, 47, 43, 28, 41,  3, 36, 41,  3, 47, 35, 32,  3, 29, 45, 42, 42, 40,
         3, 30, 39, 42, 46, 32, 47,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0]))
vocab:  {'<pad>': 0, '<sos>': 1, '<eos>': 2, ' ': 3, '!': 4, '"': 5, '$':

# Building model

In [6]:
from model.encode_decode_transformer import Transformer, TransformerConfig
from utils.trainer import Trainer, TrainerConfig
tconfig = TrainerConfig(max_epochs=2, batch_size=16, learning_rate=6e-4, grad_norm_clip=1.0, device='cuda',
                       lr_decay=True, warmup_tokens=5000, ckpt_n_print_iter=500, ckpt_path='checkpoint/transformer_vn_en')

mconfig = TransformerConfig(vocab_size=dataset.vocab_size, sequence_len=dataset.sequence_len, embed_dim=256,
                           n_block=8, n_head=8, causal=None, device=tconfig.device)

In [7]:
model = Transformer(mconfig)
trainer = Trainer(model, dataset, tconfig, test_dataset=None, collate=None)

09/27/2021 21:21:39 - INFO - model.encode_decode_transformer -   number of parameters: 1.482982e+07


In [ ]:
# load pre-trained weights
from utils.utils import pickle
model.load_state_dict(pickle(tconfig.ckpt_path)) # load

In [10]:
trainer.train()

epoch: 1 | train loss: 0.08645  | lr: 1.920000e-06:   0%|                                    | 0/15880 [00:00<?, ?it/s]

mary đang mặc một chiếc cà vạt | <sos>i like a couple of the course<eos>
tôi muốn một cuốn sách | <sos>i want some life<eos>



epoch: 1 | train loss: 0.41381  | lr: 5.997830e-04:   3%|▊                         | 500/15880 [01:31<43:23,  5.91it/s]

mất bao lâu để bay từ tokyo đến los angeles? | <sos>how much does the place is the world in boston?<eos>
đó là một cái túi mà tôi bị mất trong phòng ngày hôm qua. | <sos>that's some one of my one of this place.<eos>
mary đang mặc một chiếc cà vạt | <sos>mary is a little time in the room<eos>
tôi muốn một cuốn sách | <sos>i'd like some of the book.<eos>



epoch: 1 | train loss: 0.40981  | lr: 5.971005e-04:   6%|█▍                     | 1002/15880 [03:07<3:24:37,  1.21it/s]

...

epoch: 2 | train loss: 0.26376  | lr: 5.980486e-04:  94%|██████████████████████▋ | 15002/15880 [50:28<12:28,  1.17it/s]

mất bao lâu để bay từ tokyo đến los angeles? | <sos>how long does it from boston from boston?<eos>
đó là một cái túi mà tôi bị mất trong phòng ngày hôm qua. | <sos>it is a little disappointment that i was out of the day yesterday. yesterday.<eos>
mary đang mặc một chiếc cà vạt | <sos>mary is wearing a few house<eos>
tôi muốn một cuốn sách | <sos>i want a book.<eos>



epoch: 2 | train loss: 0.29044  | lr: 5.999730e-04:  98%|███████████████████████▍| 15501/15880 [52:00<07:18,  1.16s/it]

mất bao lâu để bay từ tokyo đến los angeles? | <sos>how long does it take to go to boston?<eos>
đó là một cái túi mà tôi bị mất trong phòng ngày hôm qua. | <sos>it was a little that i was in the room yesterday.<eos>
mary đang mặc một chiếc cà vạt | <sos>mary is wearing a few dress<eos>
tôi muốn một cuốn sách | <sos>i want a book.<eos>



epoch: 2 | train loss: 0.24582  | lr: 5.994036e-04: 100%|████████████████████████| 15880/15880 [53:08<00:00,  4.98it/s]


In [11]:
samples = ["thời tiết hôm nay thật đẹp!", 
          "xin chào", 
          "bạn đã ăn sáng chưa?"]
result = model.generate_output(samples, dataset, top_k=3)
print(result)

thời tiết hôm nay thật đẹp! | <sos>the weather is concerned! it's beautiful!<eos>
xin chào | <sos>please come along<eos>
bạn đã ăn sáng chưa? | <sos>have you eaten the morning, didn't you?<eos>

